In [ ]:
import tarfile
import os
import shutil

import glob
import numpy as np 

import tarfile

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from torch import nn, optim
from torch.autograd import Variable


def test_network(net, trainloader):

    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    dataiter = iter(trainloader)
    images, labels = dataiter.next()

    # Create Variables for the inputs and targets
    inputs = Variable(images)
    targets = Variable(images)

    # Clear the gradients from all Variables
    optimizer.zero_grad()

    # Forward pass, then backward pass, then update weights
    output = net.forward(inputs)
    loss = criterion(output, targets)
    loss.backward()
    optimizer.step()

    return True


def imshow(image, ax=None, title=None, normalize=True):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')

    return ax

In [ ]:
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar

In [ ]:
TARGET_PATH = "Images"
if os.path.exists(TARGET_PATH):
    shutil.rmtree(TARGET_PATH)

In [ ]:
fname = "images.tar"
tar = tarfile.open(fname, "r:tar")
tar.extractall()
tar.close()

In [ ]:
len(os.listdir(TARGET_PATH))

In [ ]:
for dir_name in os.listdir(TARGET_PATH):
    breed_name = dir_name.split("-")[-1].lower()
    source_dir = os.path.join(TARGET_PATH, dir_name)
    target_dir = os.path.join(TARGET_PATH, breed_name)
    shutil.move(source_dir, target_dir) 

In [ ]:
dataset = []
for filepath in glob.iglob(f'{TARGET_PATH}/**/*.jpg', recursive=True):
    breed_name = filepath.split("/")[1]
    dataset.append([filepath, breed_name])
dataset = np.array(dataset)

In [ ]:
from sklearn.model_selection import train_test_split

train_image, test_image, train_target, test_target = train_test_split(dataset[:,0], dataset[:,1], stratify=dataset[:,1])

In [ ]:
import pandas as pd
pd.Series(train_target).value_counts() / len(train_target)

In [ ]:
pd.Series(test_target).value_counts() / len(test_target)

In [ ]:
DATA_PATH = "dataset"
shutil.rmtree(DATA_PATH)
if not os.path.exists(DATA_PATH):
    os.mkdir(DATA_PATH)
    os.makedirs(os.path.join(DATA_PATH, "train"))
    os.makedirs(os.path.join(DATA_PATH, "test"))
    
    for breed_name in set(test_target):
        os.makedirs(os.path.join(DATA_PATH, "train", breed_name))
        os.makedirs(os.path.join(DATA_PATH, "test", breed_name))

In [ ]:
import shutil
for filepath, taregt_dir in zip(train_image.tolist(), train_target.tolist()):        
    filename = filepath.split("/")[-1]
    source_path = filepath
    target_dir = os.path.join(DATA_PATH, "train", taregt_dir, filename)
    shutil.copy(source_path, target_dir) 
    
for filepath, taregt_dir in zip(test_image.tolist(), test_target.tolist()):        
    filename = filepath.split("/")[-1]
    source_path = filepath
    target_dir = os.path.join(DATA_PATH, "test", taregt_dir, filename)
    shutil.copy(source_path, target_dir) 

In [ ]:
from PIL import Image
 
im = Image.open(target_dir)
im

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def process(filename: str=None) -> None:
    """
    View multiple images stored in files, stacking vertically

    Arguments:
        filename: str - path to filename containing image
    """
    image = mpimg.imread(filename)
    # <something gets done here>
    plt.figure()
    plt.imshow(image)

    
idx = np.random.choice(len(dataset), 10)

images = dataset[:, 0][idx]

for file in images:
    process(file)
    
print(dataset[:, 1][idx])

In [ ]:
from torchvision import datasets, transforms, models

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

DATASET_PATH = "dataset"
# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(DATASET_PATH + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(DATASET_PATH + '/test', transform=test_transforms)

In [ ]:
import torch
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)


In [ ]:
data_iter = iter(test_loader)

images, labels = next(data_iter)
fig, axes = plt.subplots(figsize=(10,4), ncols=4)
for ii in range(4):
    ax = axes[ii]
    imshow(images[ii], ax=ax, normalize=True)